# Data Acquisition

In [78]:
import geopandas, pandas
from pyogrio import read_dataframe

## Greenbelts

Download if not available:

In [91]:
gb_url = (
    'https://maps.communities.gov.uk/geoserver/dclg_inspire/ows'
    '?service=WFS&version=2.0.0&request=GetFeature&'
    'typeName=dclg_inspire:England_Green_Belt_2017-18_WGS84&'
    'outputFormat=json&srsName=EPSG:27700'
)

Read up:

In [94]:
%%time
gb = read_dataframe(
    gb_url,
    #where="GB_Name='Merseyside and Greater Manchester'"
)

CPU times: user 4.46 s, sys: 491 ms, total: 4.95 s
Wall time: 16.3 s


## Signatures

- Pull data if not present

In [19]:
try:
    open('signatures.gpkg').close()
except:
    ! wget https://figshare.com/ndownloader/files/30904861 -O signatures.gpkg

- Read and clip

In [70]:
def read_clip(geom, p='signatures.gpkg'):
    return geopandas.read_file(
        p, mask=geom
    ).clip(geom)

import dask.bag as db
from dask.diagnostics import ProgressBar

bag = db.from_sequence(gb.geometry.tolist()).map(read_clip)
with ProgressBar():
    clipped = pandas.concat(bag.compute())

[                                        ] | 0% Completed |  3.0s

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed
ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


[########################################] | 100% Completed |  3min 21.3s


### Store in disk

In [ ]:
clipped.to_parquet('ss_clipped.pq')